This notebook reads in government cadastral data (legal property boundaries) and user generated polygons, and then checks if the user's polygons fall outside of the cadastral bounds. Terrawise do this as aprt of their preliminary project scoping to ensure that a client has not accidentally provided boundaries for property that they do not legally own. 

This is a funciton that can be used across the wider blockbase ecosystem - that way a farmer isn't accidentally paying for satellite data for their neighbour!


In [1]:
import geopandas as gpd
import leafmap.foliumap as leafmap
import fiona

#import json

In [2]:
#import cadastral data and user polygons

cpes = gpd.read_file('data/terrawise/CPES/cpes.shp')
paddocks = gpd.read_file('data/terrawise/paddock-boundaries.shp').to_crs(cpes.crs)

In [3]:
# Create an unary union of all polygons in the first shapefile
cpes_unify = cpes.dissolve()
centroid = cpes_unify.centroid

# Apply the difference operation
difference_areas = gpd.overlay(paddocks, cpes, how='symmetric_difference') #all areas where there is only one polygon, excludes all areas where they overla.
#next, drop all polygons that are inside the CPES
outside_areas = gpd.overlay(difference_areas, cpes, how='difference')

inside_areas = paddocks.clip(cpes)

#print(outside_areas)


In [4]:
# Initialize a leafmap map
m = leafmap.Map(height="800px", width="1200px", draw_control=False, measure_control=False, fullscreen_control=False, attribution_control=True)


m.add_gdf(cpes_unify, layer_name="original cpes", info_mode=None, show=True, style={
    "stroke": True,
    "color": "green",
    "weight": 2,
    "opacity": 0.1,
    "fill": True,
    "color": "green",
    "opacity": 0.1})


m.add_gdf(paddocks, layer_name="paddocks", info_mode=None, show=True, style={
    "stroke": True,
    "color": "blue",
    "weight": 2,
    "opacity": 0.1,
    "fill": True,
    "color": "blue",
    "opacity": 0.1})


m.add_gdf(inside_areas, layer_name=" 'corrected' user boundaries", info_mode=None, show=True,style={
    "stroke": True,
    "color": "black",
    "weight": 3,
    "opacity": 1,
    "fill": False})

m.add_gdf(outside_areas, layer_name=" areas outside land parcels", info_mode=None, show=True, style={
    "stroke": True,
    "color": "red",
    "weight": 3,
    "opacity": 1,
    "fill": True})


m